In [1]:
import numpy as np

from scipy.interpolate import interp1d
from scipy.signal import find_peaks

from scipy import sparse
from scipy.sparse.linalg import spsolve

In [3]:
def baseline_als(y, lam, p, niter=10):
    L = len(y)
    D = sparse.diags([1,-2,1],[0,-1,-2], shape=(L,L-2))
    w = np.ones(L)
    for i in range(niter):
        W = sparse.spdiags(w, 0, L, L)
        Z = W + lam * D.dot(D.transpose())
        z = spsolve(Z, w*y)
        w = p * (y > z) + (1-p) * (y < z)
    return z

In [4]:
def background_flattening(y, lam=10**5, baseline_up=False, baseline_down=False):
    if baseline_up==True:
        y_baseline_up = baseline_als(y, lam=lam, p=1, niter=10)
    else:
        y_baseline_up = np.max(y)

    if baseline_down==True:
        y_baseline_down = baseline_als(y, lam=lam, p=0, niter=10)
    else:
        y_baseline_down = np.min(y)

    y_normalized = (y-y_baseline_down)/(y_baseline_up-y_baseline_down)
    return [y_normalized, y_baseline_up, y_baseline_down]

In [5]:
def from_MZI_to_detuning(data, MZI, FSR, distance, height=0.2, centralize=False):
    idx_max_MZI = find_peaks(MZI, height=1-height, distance=distance)[0]
    idx_min_MZI = find_peaks(-MZI, height=-height, distance=distance)[0]
    idx_MZI_peaks = np.sort(np.concatenate((idx_max_MZI, idx_min_MZI)))
    num_MZI_peaks = len(idx_MZI_peaks)

    detuning = interp1d(idx_MZI_peaks, np.arange(0, 0.5*FSR*num_MZI_peaks, 0.5*FSR))(np.arange(idx_MZI_peaks[0], idx_MZI_peaks[-1]+1, 1))
    if centralize==True:
        detuning = detuning - np.max(detuning)/2
    new_data = data[idx_MZI_peaks[0]:idx_MZI_peaks[-1]+1]
    new_MZI = MZI[idx_MZI_peaks[0]:idx_MZI_peaks[-1]+1]
    idx_MZI_peaks = idx_MZI_peaks - idx_MZI_peaks[0]

    return [detuning, new_data, new_MZI, idx_MZI_peaks]

In [3]:
def caique_plot(X, Y, plottype='line', annotate=False, MZI=None, idx_MZI_peaks=None, result=None):
    fig, ax = plt.subplots(figsize=(12,4))
    fig.tight_layout()

    if plottype=='scatter':
        ax.scatter(X, Y, c='black', s=50)
        if annotate:
            counter=0
            for i in X:
                ax.annotate(str(i), xy=(i, Y[counter]), size=10, c='blue')
                counter += 1
                
    elif plottype=='line':
        ax.plot(X, Y, c='black', linewidth=3)

    ax.set_xlim(X[0], X[-1])
    ax.set_xlabel("X", size=15)
    ax.set_ylabel("Y", size=15)
    ax.tick_params(direction='out', length=6, width=2, labelsize=15)
    ax.grid(True)
    if type(MZI)!=type(None):
        ax.plot(X, MZI, c="blue", linewidth=2, alpha=0.5)
    if type(idx_MZI_peaks)!=type(None):
        ax.scatter(X[idx_MZI_peaks], MZI[idx_MZI_peaks], c="blue")
    if type(result)!=type(None):
        ax.plot(X, result.init_fit, c='green', linewidth=2, linestyle='dashed', label='initial fit')
        ax.plot(X, result.best_fit, c='red', linewidth=2, linestyle='dashed', label='best fit')
        ax.legend()
        return result.params

In [6]:
def optical_mode(Δ, Δ0, κ, κe, V0):
    num = 2j*κe
    den = 2*(Δ-Δ0) + 1j*κ
    transmission = abs(1 - num/den)**2
    return V0*transmission

In [7]:
def splitted_optical_mode(Δ, Δ0, κ, κe, V0, J):
    num = 4*J**2 - (2*(Δ-Δ0) + 1j*κ)*(2*(Δ-Δ0) + 1j*(κ-2*κe))
    den = 4*J**2 - (2*(Δ-Δ0) + 1j*κ)**2
    transmission = abs(num/den)**2
    return V0*transmission

In [8]:
def optical_mode_with_fabry_perot(Δ, Δ0, κ, κe, V0, R1, R2, δ, δΔ):
    num = np.sqrt((1-R1)*(1-R2))*(2*(Δ-Δ0) + 1j*κ)*(2*(Δ-Δ0) + 1j*(κ-2*κe))
    den = np.exp(1j*δ*((Δ-Δ0) - δΔ))*np.sqrt(R1*R2)*(2*(Δ-Δ0) + 1j*(κ-2*κe))**2 - (2*(Δ-Δ0) + 1j*κ)**2
    transmission = abs(num/den)**2
    return V0*transmission

In [9]:
def splitted_optical_mode_with_fabry_perot(Δ, Δ0, κ, κe, V0, J, R1, R2, δin, δΔin, δout, δΔout):
    δ1 = δin*((Δ-Δ0) - δΔin)
    δ2 = δout*((Δ-Δ0) - δΔout)
    num = np.sqrt((1-R1)*(1-R2))*((2*(Δ-Δ0) + 1j*κ)*(2*(Δ-Δ0) + 1j*(κ-2*κe)) - 4*J**2)
    den = np.exp(2j*(δ1+δ2))*np.sqrt(R1*R2)*(4*J**2 - (2*(Δ-Δ0) + 1j*(κ-2*κe))**2) + 4j*J*κe*(np.sqrt(R1)*np.exp(2j*δ1) + np.sqrt(R2)*np.exp(2j*δ2)) - 4*J**2 + (2*(Δ-Δ0) + 1j*κ)**2
    transmission = abs(num/den)**2
    return V0*transmission

In [ ]:
def freq_disp(u, D0, D1, D2, D3):
    return D0 + D1*u + (D2/2)*u**2 + (D3/6)*u**3